In [4]:
import warnings, os
warnings.filterwarnings('ignore')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

import matplotlib.pyplot as plt
from matplotlib import colors, patches
import pandas as pd
import numpy as np
# import palettable

from astropy.table import Table
import astropy.units as u
import astropy.coordinates as coords

from tap import GaiaArchive
plt.rc("figure", dpi=120)

In [9]:
gaia = GaiaArchive()

# Functions on Gaia DataFrame

In [13]:
def make_icrs(df):
    """Make astropy coordinates from gaia_source DataFrame"""
    return coords.SkyCoord(
            ra=df.ra.values*u.deg,        
            dec=df.dec.values*u.deg,
            distance=1000./df.parallax.values*u.pc,
            pm_ra_cosdec=df.pmra.values*u.mas/u.year,
            pm_dec=df.pmdec.values*u.mas/u.year,
            radial_velocity=df.radial_velocity.values*u.km/u.s)

In [14]:
def add_xv(df, frame, unit=u.pc):
    """
    Add cartesian coordinates x, y, z, vx, vy, vz for a given `frame`
    
    df : pd.DataFrame
        Gaia DR2 data
    frame : astropy coordinate frame
        Frame to calculate coordinates in
    
    Returns df with x, y, z, vx, vy, vz columns added.
    """
    df = df.copy()
    c = make_icrs(df).transform_to(frame)
    df['x'], df['y'], df['z'] = c.cartesian.xyz.to(u.pc).value
    df['vx'], df['vy'], df['vz'] = c.velocity.d_xyz.value
    return df

In [15]:
def extract_a_g_error(df):
    """
    Extract lerr and uerr of a_g from percentiles.
    Returns df with a_g_lerr, a_g_uerr columns added.
    """
    df = df.copy()
    lerr = df['a_g_val'] - df['a_g_percentile_lower']
    uerr = df['a_g_percentile_upper'] - df['a_g_val']
    df['a_g_lerr'], df['a_g_uerr'] = lerr, uerr
    return df

In [16]:
def add_gMag(df):
    df = df.copy()
    df['gMag'] = df['phot_g_mean_mag'] + 5*np.log10(df['parallax']) - 10
    return df

In [17]:
def flag_good_phot(df):
    df = df.copy()
    good_phot = ((df.phot_bp_rp_excess_factor > 1+0.015*df.bp_rp**2) & (df.phot_bp_rp_excess_factor < 1.3+0.06*df.bp_rp**2))
    df['good_phot'] = good_phot
    return df

In [ ]:
def flag_excess(df):
    df = df.copy()
    ee = np.zeros(len(df)).astype(bool)
    ee[(df.bp_rp>1.8)\
       & ((df.bp_rp-1.8)*0.65<df.astrometric_excess_noise)\
       &(df.astrometric_excess_noise_sig>3)] = True
    ee[(df.bp_rp<1.8) \
       & (df.astrometric_excess_noise>0)\
       & (df.astrometric_excess_noise_sig>3)] = True
    df['excess_excess'] = ee
    return df

In [20]:
def table_to_pandas(t):
    """Convert astropy table to pandas dataframe.
    When source_id is converted to float, significant digits are lost, which
    result in duplicated source_id which is actually unique. To prevent this,
    when the column dtype is int and there is no missing values, keep dtype int
    instead of converting to nan.
    NOTE: This conversion happens in the first place because np.nan is not supported for int types.
    """
    from collections import OrderedDict
    out = OrderedDict()
    for name, column in t.columns.items():
        if column.dtype.kind in ['i', 'u']:
            if column.mask.sum() == 0:
                out[name] = column.astype(int)
            else:
                out[name] = column.astype(float).filled(np.nan)
        elif column.dtype.kind in ['f', 'c']:
            out[name] = column.filled(np.nan)
        else:
            out[name] = column.astype(object).filled(np.nan)


        if out[name].dtype.byteorder not in ('=', '|'):
            out[name] = out[name].byteswap().newbyteorder()

    return pd.DataFrame(out)

In [1]:
def cached(cachefile):
    """
    A simple cache decorator that saves return value of the function to `cachefile`.
    If the file exists, the function does not run and reads output from the cached file.
    """
    import os
    import pickle

    def decorator(fn):  # define a decorator for a function "fn"
        def wrapped(*args, **kwargs):   # define a wrapper that will finally call "fn" with all arguments            
            # if cache exists -> load it and return its content
            if os.path.exists(cachefile):
                    with open(cachefile, 'rb') as cachehandle:
                        print("using cached result from '%s'" % cachefile)
                        return pickle.load(cachehandle)

            # execute the function with all arguments passed
            res = fn(*args, **kwargs)

            # write to cache file
            with open(cachefile, 'wb') as cachehandle:
                print("saving result to cache '%s'" % cachefile)
                pickle.dump(res, cachehandle)

            return res

        return wrapped

    return decorator   # return this "customized" decorator that uses "cachefile"